# Model inference

Add local paths to system in order to use local modules.

In [1]:
import sys
import os

# Get the current directory of the notebook
current_dir = os.getcwd()

# Construct the path to the parent directory
parent_dir = os.path.join(current_dir, "..")

# Add the parent directory to sys.path
sys.path.append(parent_dir)

Load packages.

In [2]:
from glob import glob

import numpy as np

from src.fsl import build_siamese_network
from src.dataset import prepare_image

## Load a trained model

In [3]:
loaded_model = build_siamese_network()
loaded_model.load_weights("../models/siamese.weights.h5")

2025-09-09 19:30:57.629697: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-09-09 19:30:57.629716: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-09-09 19:30:57.629719: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-09-09 19:30:57.629733: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-09 19:30:57.629745: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
loaded_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 64)        │    189,504 │ input_layer[0][0… │
│ (Sequential)        │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 1)         │          0 │ sequential[0][0], │
│                     │                   │            │ sequential[1][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │          2 │ lambda[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 189,506 (740.26 KB)

 Trainable params: 189,506 (740.26 KB)

 Non-trainable params: 0 (0.00 B)

## Load query images

In [5]:
# Load query images
query_folder = "../data/query/*.png"
query_images = []
class_names = []
for img_path in glob(query_folder):
    class_name = img_path.split("/")[-1][:-4]
    img = prepare_image(img_path, target_size=(256, 256))
    img = np.expand_dims(img, axis=0)
    query_images.append(img)
    class_names.append(class_name)

## Prepare image for inference

In [6]:
# Prepare cilinder
dataset_folder = "../data/dataset/cilinder/*.png"
random_cilinder = np.random.choice(glob(dataset_folder))
cilinder_img = prepare_image(random_cilinder, target_size=(256, 256))
cilinder_img = np.expand_dims(cilinder_img, axis=0)

In [7]:
# Prepare ticket
dataset_folder = "../data/dataset/ticket/*.png"
random_ticket = np.random.choice(glob(dataset_folder))
ticket_img = prepare_image(random_ticket, target_size=(256, 256))
ticket_img = np.expand_dims(ticket_img, axis=0)

## Run inference with images

In [8]:
for image, class_name in zip(query_images, class_names):
    score = loaded_model.predict([cilinder_img, image])
    print(f"Score for cilinder: {score[0]} | Query: {class_name}")

2025-09-09 19:31:01.040471: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 444ms/step
Score for cilinder: [0.14109139] | Query: ticket
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Score for cilinder: [0.81894785] | Query: cylinder


In [9]:
for image, class_name in zip(query_images, class_names):
    score = loaded_model.predict([ticket_img, image])
    print(f"Score for ticket: {score[0]} | Query: {class_name}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Score for ticket: [0.67334205] | Query: ticket
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Score for ticket: [0.0698039] | Query: cylinder


---